In [76]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Module to see files and folders in directories
from scipy import stats
from scipy import optimize

In [77]:
# Defining the parameters:
mu1   =  1.614
sig1  =  0.0003
mu2   =  0.23
sig2  =  0.0007
mu3   =  0.01549
sig3  =  0.00004
mu4   =  0.00603
sig4  =  0.00008
rho12 =  0 

In [78]:
if not (-1.0 <= rho12 <= 1.0): 
    raise ValueError(f"Correlation factor not in interval [-1,1], as it is {rho12:6.2f}")

In [79]:
from IPython.core.display import Latex

def lprint(*args,**kwargs):
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)

def myDiff(formula):
    return sqrt((formula.diff(A) * dA)**2 + (formula.diff(E) * dE)**2 + (formula.diff(D) * dD)**2 + (formula.diff(R) * dR)**2)

def myDiffWithCorr(formula, name = "", printNow = False):
    dd = sqrt((formula.diff(A) * dA)**2 + 2*(formula.diff(A)*formula.diff(E)*(sigCorr**2)) 
              + 2*(formula.diff(A)*formula.diff(D)*(sigCorr**2)) + 2*(formula.diff(A)*formula.diff(R)*(sigCorr**2))
              + (formula.diff(E) * dE)**2 +  2*(formula.diff(E)*formula.diff(D)*(sigCorr**2)) 
              + 2*(formula.diff(E)*formula.diff(R)*(sigCorr**2)) + (formula.diff(D) * dD)**2 
              + 2*(formula.diff(D)*formula.diff(R)*(sigCorr**2)) + (formula.diff(R) * dR)**2 
             )
    if(printNow):
        lprint(latex(Eq(symbols('sigma_'+name), dd)))
    fd = lambdify((A,dA,E,dE,D,dD,R,dR,sigCorr),dd)
    return dd, fd
    
def diff_and_print(formula, name = ""):
    # Calculate uncertainty and print original relation/formula and the uncertainty
    dd = myDiff(formula)
    lprint(latex(Eq(symbols(name),formula)))
    lprint(latex(Eq(symbols('sigma_'+name), dd)))
    
def lambdifyFormula(formula, *args, name = ""):
    # Turn expression into numerical functions 
    f = lambdify((A,E,D,R),formula)
    d = myDiff(formula)
    fd = lambdify((A,dA,E,dE,D,dD,R,dR),d)
    return f, fd

In [80]:
# Import SymPy: 
from sympy import * 
    
# Define variables:
G,A,E,D,R = symbols("G,A,E,D,R")
dG,dA,dE,dD,dR = symbols("sigma_G, sigma_A, sigma_E, sigma_D, sigma_R")

# Define relations:
# Diagonal
G = A/E * (1+(2/5)* (D**2/(D**2 - R**2)))

# Try writing a simple function to not repeat yourself! (See cell above)
diff_and_print(G,"G")

dG = myDiff(G)

# Turn expressions into numerical functions 
fG, fdG = lambdifyFormula(G,"G")

# Define values and their errors
vA, vdA = mu1,sig1
vE, vdE = mu2,sig2
vD, vdD = mu3,sig3
vR, vdR = mu4,sig4

# Numerically evaluate expressions and print 
vG = fG(vA,vE,vD,vR)
vdG = fdG(vA,vdA,vE,vdE,vD,vdD,vR,vdR)


#Adding correlations (and also derivation, printing and lambdifying)
sigCorr = symbols("sigma_AEDR")
rho = symbols("rho_AEDR")

dG, fdG = myDiffWithCorr(G, "G", True)

sCorr = sqrt(rho*dA*dE*dD*dR)
fSC = lambdify((rho,dA,dE,dD,dR),sCorr)

vSigmaCorr = fSC(rho12,vdA,vdE,vdD,vdR)

# Numerically evaluate expressions and print 
vdG = fdG(vA,vdA,vE,vdE,vD,vdD,vR,vdR,vSigmaCorr)

lprint(fr'G = ({vG:.3f} \pm {vdG:.4f})\,\mathrm{{m}}')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>